## 构建知识库
> 基于向量数据库构建供应链计划知识库

#### 参考文档
1. https://python.langchain.com/docs/tutorials/retrievers/
2. https://zhuanlan.zhihu.com/p/260083265

#### 向量检索相关知识
问题集：
1. 如何更方便和快速的构建知识库，尤其是自动化收集知识？
2. 为什么要把文本向量化？常见的向量化方案是什么？
3. 如何度量文本的相似性？
4. 向量检索如何实现快速检索？即构建向量索引的方法是什么？

#### 核心步骤
1. 加载原始数据（非结构化的文本数据）并预处理
3. 存储处理后的文档集到向量数据库

#### 第一步：加载原始数据并预处理
数据来源CSV文件，其它类型的数据加载可以参考文档：https://python.langchain.com/docs/how_to/#document-loaders

RecursiveCharacterTextSplitter -> TextSplitter -> DocumentTransformer

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 相对路径
csv_file_path = "./knowledge-documents.csv"
loader = CSVLoader(
    file_path=csv_file_path,
    content_columns=["知识项"],
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        # "fieldnames": ["知识项"]
    }
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = loader.load_and_split(text_splitter)

#### 第二步：构建向量数据库
使用Ollama拉取Embedding模型nomic-embed-text（具体指令`ollama pull nomic-embed-text`），拉取成功后即可使用.

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_store = Chroma(collection_name="supply-chain-knowledge-vectors", embedding_function=embeddings, persist_directory="./.chroma_db")

In [4]:
vector_store.add_documents(documents=documents)

['be8f1287-14a2-40e4-806b-9a7e667e523e',
 '14d38f9a-1a4f-4e98-ac13-42c3dd9d433e',
 '34643d8b-31d0-4a5b-92b8-9fe7c4635e89',
 'a93025f8-f3df-4be6-a91e-f16ce1c5ecf4',
 '03becd8b-d0b2-44aa-a4f1-95e9ea1aa07a',
 '8bb7609e-6761-4109-8ab9-a751ac583380',
 'd91057e2-ceef-40f6-81c1-8bf33a3d0443',
 '9c85ce3b-5209-423e-83d1-4322176eb1d4']

In [5]:
vector_store.similarity_search("IBP集成业务计划")

[Document(id='9c85ce3b-5209-423e-83d1-4322176eb1d4', metadata={'row': 7, 'source': './knowledge-documents.csv'}, page_content='知识项: IBP不仅是规划工具，更是战略管理平台，通过整合数据、流程与文化，帮助企业实现从被动响应到主动规划的数字化转型。其核心价值在于提升敏捷性、优化资源配置，从而在动态市场中保持竞争力。'),
 Document(id='d91057e2-ceef-40f6-81c1-8bf33a3d0443', metadata={'row': 6, 'source': './knowledge-documents.csv'}, page_content='知识项: IBP（集成业务计划，Integrated Business Planning）是一种战略性、跨职能的业务规划方法，旨在通过整合企业内部的战略、财务、运营等环节，实现资源优化和高效协同。IBP的定义与内涵：滚动规划计划、三位一体整合。IBP的核心目标：战略与执行对齐、动态响应市场变化、资源优化与成本控制。IBP的实施流程：数据整合与统一视图、跨部门协作与决策、动态优化与模拟分析。'),
 Document(id='a93025f8-f3df-4be6-a91e-f16ce1c5ecf4', metadata={'row': 3, 'source': './knowledge-documents.csv'}, page_content='知识项: 库存计划的实现方案有：基于预测的库存计划、ABC分类法、经济订货批量（EOQ）模型、供应商管理库存（VMI）、联合库存管理（JMI）、寄售库存。基于预测的库存计划涉及一些重要的参数：再订货点、目标库存水位、Smin/Smax、安全库存。'),
 Document(id='03becd8b-d0b2-44aa-a4f1-95e9ea1aa07a', metadata={'row': 4, 'source': './knowledge-documents.csv'}, page_content='知识项: S&OP是一种将销售、运营、财务等多部门计划进行整合的管理流程。它通过对市场需求、供应能力、库存水平、财务预算等多方面因素